# Neural Network Model for MNIST Dataset

### 패키지

In [1]:
import numpy as np
import math

import tensorflow as tf

import keras
from keras import backend as K
from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical

from sklearn.manifold import TSNE
from skimage import io

import theano

import json

Using TensorFlow backend.


In [2]:
 # Functions 
def get_hot_idx(arr):
    return arr.index(max(arr))

def save_to_json_file(data, filename):
    with open(filename, 'w') as outfile:
        json.dump(data, outfile)
    print(filename + ' 저장완료')
    
def get_round_array(array, decimal):
    return [round(e, decimal) for e in array]

def get_activations(model, layer, X_batch):
    get_activations = K.function([model.layers[0].input, K.learning_phase()], [model.layers[layer].output,])
    activations = get_activations([X_batch,0])
    return activations

def get_arr_from_json_file(filename):
    input_file = open (filename)
    return json.load(input_file)
    
current_milli_time = lambda: int(round(time.time() * 1000))

### 데이터셋 로드

In [3]:
image_width = 28
image_height = 28
num_of_features = image_width * image_height

num_of_class = 10
num_of_trainset = 60000
num_of_testset = 10000

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = X_train.reshape(num_of_trainset, num_of_features).astype('float32') / 255.0
X_test = X_test.reshape(num_of_testset, num_of_features).astype('float32') / 255.0
Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)

In [4]:
# 로컬 테스트 데이터 로드
images = np.zeros((10000, 784))
for real in range(10):
    for idx in range(1, 1001):
        file = '../../data/mnist/images/'+ str(real) + '/' + str(real) + '_' + str(idx) +'.png'
        image = np.ndarray.flatten(io.imread(file)) / 255.0
        image = np.array([1 - pixel for pixel in image])
        images[real * 1000 + idx - 1] = image

### 모델 구성

In [294]:
# 모델 구축 
model = Sequential()
model.add(Dense(units=256, input_dim=num_of_features, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=num_of_class, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

### 모델 학습

In [295]:
model.fit(X_train, Y_train, epochs=5, batch_size=32)

Epoch 1/5
60000/60000 [==============================] - 8s 131us/step - loss: 2.0212 - acc: 0.2215
Epoch 2/5
60000/60000 [==============================] - 6s 106us/step - loss: 0.8309 - acc: 0.7131
Epoch 3/5
37120/60000 [=================>............] - ETA: 2s - loss: 0.3723 - acc: 0.9120

KeyboardInterrupt: 

### 테스트 셋 결과 확인

In [296]:
loss_and_metrics = model.evaluate(X_test, Y_test, batch_size=32)
print('loss_and_metrics : ' + str(loss_and_metrics))

10000/10000 [==============================] - 1s 91us/step
loss_and_metrics : [0.33707622410058974, 0.9266]


## 내 데이터로 테스트

In [297]:
# 예측
pred_proba =  model.predict_proba(images).tolist()

In [298]:
# 성능 확인 및 저장
correct = 0
performances = {
    "accuracy": 0,
    "recall": [0] * 10,
    "precision": [0] * 10
}
truePredict = [0] * 10
numOfPredict = [0] * 10
predicts = []

for i in range(len(pred_proba)):
    prob = pred_proba[i]
    pred = get_hot_idx(prob)
    real = i // 1000
    predicts.append(dict({
        "real": real,
        "pred": pred,
        "prob": [round(e, 4) for e in prob]
    }))
    numOfPredict[pred] = numOfPredict[pred] + 1
    if pred is real:
        truePredict[real] = truePredict[real] + 1
        correct = correct + 1

performances["accuracy"] = correct / 10000
performances["recall"] = [round(truePredict[i] / 1000, 4) for i in range(10) ]
performances["precision"] = [round(truePredict[i] / numOfPredict[i], 4) for i in range(10)]

print(performances)

{'accuracy': 0.92, 'recall': [0.959, 0.974, 0.967, 0.906, 0.917, 0.888, 0.906, 0.898, 0.873, 0.912], 'precision': [0.9697, 0.9402, 0.8293, 0.9273, 0.9348, 0.9397, 0.9869, 0.9677, 0.8713, 0.862]}


### 차원축소

1) 마지막 직전의 레이어에서 activation values를 뽑는다.

2) t-SNE로 차원을 축소한다.

In [299]:
samples = []
idxs = get_arr_from_json_file('./sample_image_idxs.json')
for idx in idxs:
    samples.append(images[idx])
print(len(samples))

500


In [300]:
extracted_features = get_activations(model, -2, samples)[0]
y = TSNE(n_components=2).fit_transform(extracted_features)
sne_map = []
for e in y:
    sne_map.append({
        "x": round(e[0], 2),
        "y": round(e[1], 2)
    })

In [301]:
model_data = {
    "model_name": 'Deep Neural Network',
    "short_name": 'DNN-15',
    "description": '15개의 히든 레이어를 가지는 심층 신경망 모델, epochs=5, batch_size=32',
    "performance": {
        "accuracy": performances["accuracy"],
        "recall": performances["recall"],
        "precision": performances["precision"]
    },
    "predict": predicts,
    "t-sne": sne_map
}

In [302]:
with open('mnist_dnn-15.json', 'w', encoding='utf-8') as f:
    json.dump(str(model_data), f, ensure_ascii=False, indent=4)